d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# 3.6 Writing Data

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Employ the writing design pattern
* Control partitions for database writes using REPARTITION and COALESCE hints

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Writing From Spark<br>

-sandbox
Writing to a database in Spark differs from other tools largely due to its distributed nature. There are a number of variables that can be tweaked to optimize performance, largely relating to how data is organized on the cluster. Partitions are the first step in understanding performant database connections.

**A partition is a portion of your total data set,** which is divided into many of these portions so Spark can distribute your work across a cluster. 

The other concept needed to understand Spark's computation is a slot (also known as a core). **A slot/core is a resource available for the execution of computation in parallel.** In brief, a partition refers to the distribution of data while a slot refers to the distribution of computation.

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/partitions-and-cores.png" style="height: 400px; margin: 20px"/></div>

As a general rule of thumb, the number of partitions should be a multiple of the number of cores. For instance, with 5 partitions and 8 slots, 3 of the slots will be underutilized. With 9 partitions and 8 slots, a job will take twice as long as it waits for the extra partition to finish.

Import the dataset.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV
USING CSV 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-truncated.csv",
    header "true",
    sep ":"
  )

Switch to the Python API to run write commands.  You can do this `%python` and then run `sql(" <My Normal SQL Query> ")`

In [0]:
%python
df = sql("SELECT * FROM fireCallsCSV LIMIT 10 ")

display(df)

Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,On Scene DtTm,Transport DtTm,Hospital DtTm,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,Original Priority,Priority,Final Priority,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,Location,RowID
001030118,E08,00030625,Medical Incident,04/12/2000,04/12/2000,04/12/2000 09:27:45 PM,04/12/2000 09:28:58 PM,04/12/2000 09:29:21 PM,04/12/2000 09:31:26 PM,04/12/2000 09:32:34 PM,null,null,Other,04/12/2000 09:45:28 PM,4TH ST/CHANNEL ST,SF,null,B03,08,2226,3,3,3,false,null,1,ENGINE,1,3,6,None,"(37.7750268633971, -122.392346204303)",001030118-E08
001030122,M18,00030630,Medical Incident,04/12/2000,04/12/2000,04/12/2000 09:31:55 PM,04/12/2000 09:33:48 PM,04/12/2000 09:34:10 PM,04/12/2000 09:35:59 PM,04/12/2000 09:45:22 PM,null,null,Other,04/12/2000 09:49:52 PM,1800 Block of IRVING ST,SF,94122,B08,22,7424,1,1,2,false,null,1,MEDIC,1,8,4,Sunset/Parkside,"(37.763482287794, -122.477678638767)",001030122-M18
001030154,M36,00030662,Medical Incident,04/12/2000,04/12/2000,04/12/2000 10:43:54 PM,04/12/2000 10:45:53 PM,04/12/2000 10:49:59 PM,04/12/2000 10:50:35 PM,04/12/2000 10:53:18 PM,04/12/2000 11:11:36 PM,04/12/2000 11:22:17 PM,Other,04/12/2000 11:42:43 PM,0 Block of SOUTH VAN NESS AVE,SF,94103,B02,36,5117,1,1,2,false,null,1,MEDIC,1,2,6,Mission,"(37.7741251002903, -122.418810211803)",001030154-M36
001040007,E12,00030697,Structure Fire,04/13/2000,04/12/2000,04/13/2000 12:19:54 AM,04/13/2000 12:29:24 AM,04/13/2000 12:29:35 AM,04/13/2000 12:31:25 AM,04/13/2000 12:32:36 AM,null,null,Other,04/13/2000 12:33:18 AM,CLAYTON ST/PARNASSUS AV,SF,94117,B05,12,5151,3,3,3,true,null,1,ENGINE,1,5,5,Haight Ashbury,"(37.7651387353822, -122.44763462758)",001040007-E12
001040021,M14,00030711,Medical Incident,04/13/2000,04/12/2000,04/13/2000 01:17:25 AM,04/13/2000 01:18:44 AM,04/13/2000 01:20:02 AM,04/13/2000 01:21:40 AM,04/13/2000 01:24:05 AM,04/13/2000 01:56:02 AM,04/13/2000 02:33:33 AM,Other,04/13/2000 02:40:25 AM,500 Block of 38TH AVE,SF,94121,B07,34,7255,3,3,3,true,null,1,MEDIC,1,7,1,Outer Richmond,"(37.778489948235, -122.498662035969)",001040021-M14
001040061,M43,00030749,Medical Incident,04/13/2000,04/12/2000,04/13/2000 07:51:29 AM,04/13/2000 07:55:35 AM,04/13/2000 07:55:54 AM,04/13/2000 07:59:58 AM,null,04/13/2000 08:16:30 AM,04/13/2000 08:28:37 AM,Other,04/13/2000 09:26:54 AM,200 Block of MADRID ST,SF,94112,B09,43,0613,3,3,3,true,null,1,MEDIC,3,9,11,Excelsior,"(37.7255316247491, -122.429925994016)",001040061-M43
001040079,E10,00030766,Alarms,04/13/2000,04/13/2000,04/13/2000 09:31:19 AM,04/13/2000 09:33:04 AM,04/13/2000 09:34:10 AM,04/13/2000 09:35:52 AM,04/13/2000 09:37:59 AM,null,null,Other,04/13/2000 09:39:36 AM,2800 Block of BROADWAY,SF,94123,B04,10,4226,3,3,3,false,null,1,ENGINE,1,4,2,Pacific Heights,"(37.7931736175933, -122.444028632879)",001040079-E10
001040143,M43,00030832,Medical Incident,04/13/2000,04/13/2000,04/13/2000 01:01:56 PM,04/13/2000 01:04:12 PM,04/13/2000 01:13:08 PM,null,04/13/2000 01:29:19 PM,04/13/2000 01:36:34 PM,null,Other,04/13/2000 01:16:20 PM,2500 Block of OCEAN AVE,SF,94132,B08,19,8452,1,1,2,true,null,1,MEDIC,1,8,7,West of Twin Peaks,"(37.7314853147957, -122.472647880057)",001040143-M43
001040170,T16,00030855,Structure Fire,04/13/2000,04/13/2000,04/13/2000 02:09:54 PM,04/13/2000 02:12:27 PM,04/13/2000 02:15:00 PM,null,null,null,null,Other,04/13/2000 02:22:01 PM,POLK ST/UNION ST,SF,94109,B04,04,3131,3,3,3,false,null,1,TRUCK,2,4,3,Russian Hill,"(37.7987615790944, -122.422336952094)",001040170-T16
001040233,E48,00030914,Alarms,04/13/2000,04/13/2000,04/13/2000 05:23:03 PM,04/13/2000 05:23:58 PM,04/13/2000 05:25:02 PM,null,04/13/2000 05:29:16 PM,null,null,Other,04/13/2000 05:52:48 PM,CALL BOX: FS TI,TI,94130,B03,48,2931,3,3,3,false,null,1,ENGINE,1,None,6,T

Write using the `.write(" < path> ")` method on the DataFrame.

In [0]:
%python
df.write.mode("OVERWRITE").csv(username + "/fire-calls.csv")

Take a look at the file we just wrote.

In [0]:
%python
dbutils.fs.ls(username + "/fire-calls.csv")

Notice that it wrote a number of different parts, one for each partition.  Now let's get a sense for how to check how many partitions our data has, which once again is in Python.

In [0]:
%python
df.rdd.getNumPartitions()

Out[9]: 1

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Controlling Concurrency<br>

In the context of JDBC database writes, **the number of partitions determine the number of connections used to push data through the JDBC API.** There are two ways to control this parallelism:  

| Hint | Transformation Type | Use | Evenly distributes data across partitions? |
| :----------------|:----------------|:----------------|:----------------| 
| `SELECT /*+ COALESCE(n) */`   | narrow (does not shuffle data) | reduce the number of partitions | no |
| `SELECT /*+ REPARTITION(n) */`| wide (includes a shuffle operation) | increase the number of partitions | yes |

Create a new temporary view that coalesces all of our data to a single partition.  This can be done using a hint.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV1p
  AS
SELECT /*+ COALESCE(1) */ * 
FROM fireCallsCSV

Now check the number of partitions.

In [0]:
%python
sql("SELECT * FROM fireCallsCSV1p").rdd.getNumPartitions()

Out[10]: 1

We now have a single partition.  What if we want more partitions?  Let's use a `REPARTITION` hint to evenly distribute our data across 8 partitions.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsCSV12p
  AS
SELECT /*+ REPARTITION(12) */ * 
FROM fireCallsCSV

In [0]:
%python
sql("SELECT * FROM fireCallsCSV12p").rdd.getNumPartitions()

Out[12]: 12

Now you can save the results.

In [0]:
%python
sql("SELECT * FROM fireCallsCSV12p").write.mode("OVERWRITE").csv(username + "/fire-calls-repartitioned.csv")

Click the arrow next to `Spark Jobs` under the following code cell in order to see a breakdown of the job you triggered. Click the next arrow to see a breakdown of the stages.

When you repartitioned the DataFrame to 12 partitions, 12 stages were needed, one to write each partition of the data.

View the results.

In [0]:
%python
dbutils.fs.ls(username + "/fire-calls-repartitioned.csv")

## Summary
- Alter the number of concurrent writes using `SELECT /*+ COALESCE(n) */` or `SELECT /*+ REPARTITION(n) */`
- Note that `coalesce` means something different in ANSI SQL (that is, to return the first non-null value)
- Files like CSV and Parquet might look like single files, but Spark saves them as a folder of different parts

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>